In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<h2>Get Top 10 Cryptos History</h2>

In [2]:
from coinMarketCap import getTopHistory

dfs = getTopHistory(limit=10, days=90)


<h2>Calc Statistics</h2>

In [3]:
statColumns = [
    'avg[(max - min)/(max + min)]',
    'avg[(max - open)/ open ]',
    'avg[(min - open)/ open ]',
    'std[(max - min)/(max + min)]',
    'std[(max - open)/ open ]',
    'std[(min - open)/ open ]',
]
cryptoStat = pd.DataFrame(columns=statColumns)

for coin in dfs:
    df = dfs[coin]
    data = []

    data.append(((df.high - df.low)/(df.high + df.low)).mean())
    data.append(((df.high - df.open)/df.open).mean())
    data.append(((df.low - df.open)/df.open).mean())

    data.append(((df.high - df.low)/(df.high + df.low)).std())
    data.append(((df.high - df.open)/df.open).std())
    data.append(((df.low - df.open)/df.open).std())

    cryptoStat = cryptoStat.append(pd.DataFrame([data],columns=statColumns,index=[coin]))

cryptoStat.T

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC
avg[(max - min)/(max + min)],0.024914,0.032277,0.032874,0.058828,0.000584,0.038860,0.041278,0.051509,0.043373,0.000627
avg[(max - open)/ open ],0.026090,0.032761,0.034146,0.072902,0.000614,0.041519,0.044387,0.057841,0.047026,0.000591
avg[(min - open)/ open ],-0.023691,-0.031598,-0.031419,-0.047295,-0.000555,-0.036311,-0.038135,-0.045664,-0.040241,-0.000661
std[(max - min)/(max + min)],0.012712,0.016772,0.018547,0.031797,0.000246,0.024127,0.028034,0.029175,0.028754,0.000685
std[(max - open)/ open ],0.022451,0.026922,0.029323,0.066954,0.000400,0.043178,0.046299,0.050796,0.054685,0.000374
std[(min - open)/ open ],0.025292,0.032206,0.034764,0.041258,0.000338,0.040723,0.046659,0.046925,0.042619,0.001360


<h2>Calc fraction of days</h2>

In [4]:
statColumns_2 = [
    '[max - min] / open >  5 %',
    '[max - min] / open > 10 %',
    '[max - min] / open > 15 %',
    '[max - min] / open > 20 %',
]
cryptoStat_2 = pd.DataFrame(columns=statColumns_2)

for coin in dfs:
    df = dfs[coin]
    data = []
    c = (df.high - df.low)/df.open
    data.append(round((len(c[c >= 0.05]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.10]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.15]) / len(c) * 100),1))
    data.append(round((len(c[c >= 0.20]) / len(c) * 100),1))
    
    cryptoStat_2 = cryptoStat_2.append(pd.DataFrame([data],columns=statColumns_2,index=[coin]))

cryptoStat_2.T

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC
[max - min] / open > 5 %,40.4,61.8,57.3,94.4,0.0,68.5,71.9,89.9,73.0,0.0
[max - min] / open > 10 %,4.5,10.1,14.6,49.4,0.0,25.8,25.8,41.6,30.3,0.0
[max - min] / open > 15 %,1.1,1.1,1.1,22.5,0.0,7.9,7.9,13.5,7.9,0.0
[max - min] / open > 20 %,0.0,1.1,1.1,13.5,0.0,1.1,4.5,6.7,3.4,0.0


percentage of days

<h2>Count of days when min == open AND max == open</h2>

In [5]:
statColumns_3 = [
    'min == open',
    'max == open',
]
cryptoStat_3 = pd.DataFrame(columns=statColumns_3)

for coin in dfs:
    df = dfs[coin]
    data = []
    data.append(len(df[df['low'] == df['open']]))
    data.append(len(df[df['high'] == df['open']]))
    
    cryptoStat_3 = cryptoStat_3.append(pd.DataFrame([data],columns=statColumns_3,index=[coin]))

cryptoStat_3["sum"] = cryptoStat_3.sum(axis=1)
cryptoStat_3.T

,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC
min == open,0,0,1,3,0,0,1,0,0,0
max == open,1,1,0,1,0,2,1,2,0,1
sum,1,1,1,4,0,2,2,2,0,1


<h2>Fraction of days with Condition: min < open AND max > open</h2>

In [6]:
statColumns_4 = [
    '[max - min] / open >  5 %',
    '[max - min] / open > 10 %',
    '[max - min] / open > 15 %',
    '[max - min] / open > 20 %',
]
cryptoStat_4 = pd.DataFrame(columns=statColumns_4)

for coin in dfs:
    df = dfs[coin]
    total = len(df)
    df = df[(df['low'] < df['open']) & (df['high'] > df['open'])]
    data = []
    c = (df.high - df.low)/df.open
    data.append(round((len(c[c >= 0.05]) / total * 100),1))
    data.append(round((len(c[c >= 0.10]) / total * 100),1))
    data.append(round((len(c[c >= 0.15]) / total * 100),1))
    data.append(round((len(c[c >= 0.20]) / total * 100),1))
    
    cryptoStat_4 = cryptoStat_4.append(pd.DataFrame([data],columns=statColumns_4,index=[coin]))

cryptoStat_4.T


,BTC,ETH,BNB,SOL,USDT,ADA,XRP,DOT,DOGE,USDC
[max - min] / open > 5 %,40.4,61.8,56.2,89.9,0.0,66.3,69.7,87.6,73.0,0.0
[max - min] / open > 10 %,4.5,10.1,14.6,48.3,0.0,25.8,25.8,41.6,30.3,0.0
[max - min] / open > 15 %,1.1,1.1,1.1,21.3,0.0,7.9,7.9,13.5,7.9,0.0
[max - min] / open > 20 %,0.0,1.1,1.1,13.5,0.0,1.1,4.5,6.7,3.4,0.0


<h2>Final Result for Comparison</h2>

In [7]:
pd.DataFrame(
    cryptoStat_2.T.append(cryptoStat_4.T).values,
    columns=pd.Index(list(dfs.keys())),
    index=pd.MultiIndex.from_product([['All days', 'max > open AND min < open'], statColumns_2]))


BTC   ETH   BNB   SOL  \
All days                  [max - min] / open >  5 %  40.4  61.8  57.3  94.4   
                          [max - min] / open > 10 %   4.5  10.1  14.6  49.4   
                          [max - min] / open > 15 %   1.1   1.1   1.1  22.5   
                          [max - min] / open > 20 %   0.0   1.1   1.1  13.5   
max > open AND min < open [max - min] / open >  5 %  40.4  61.8  56.2  89.9   
                          [max - min] / open > 10 %   4.5  10.1  14.6  48.3   
                          [max - min] / open > 15 %   1.1   1.1   1.1  21.3   
                          [max - min] / open > 20 %   0.0   1.1   1.1  13.5   

                                                     USDT   ADA   XRP   DOT  \
All days                  [max - min] / open >  5 %   0.0  68.5  71.9  89.9   
                          [max - min] / open > 10 %   0.0  25.8  25.8  41.6   
                          [max - min] / open > 15 %   0.0   7.9   7.9  13.5   
                          [max - min] / open > 20 %   0.0   1.1   4.5   6.7   
max > open AND min < open [max - min] / open >  5 %   0.0  66.3  69.7  87.6   
                          [max - min] / open > 10 %   0.0  25.8  25.8  41.6   
                          [max - min] / open > 15 %   0.0   7.9   7.9  13.5   
                          [max - min] / open > 20 %   0.0   1.1   4.5   6.7   

                                                     DOGE  USDC  
All days                  [max - min] / open >  5 %  73.0   0.0  
                          [max - min] / open > 10 %  30.3   0.0  
                          [max - min] / open > 15 %   7.9   0.0  
                          [max - min] / open > 20 %   3.4   0.0  
max > open AND min < open [max - min] / open >  5 %  73.0   0.0  
                          [max - min] / open > 10 %  30.3   0.0  
                          [max - min] / open > 15 %   7.9   0.0  
                          [max - min] / open > 20 %   3.4   0.0